In [ ]:
type [<Struct>] Position = { X : int; Y : int }
type Sensor = { Sensor : Position; Beacon : Position }
let parseLine (line : string) =
    let bits = 
        line.Split[| '='; ','; ':' |] 
    { Sensor = { X = int bits[1]; Y = int bits[3] }; Beacon = { X = int bits[5]; Y = int bits[7] } }

let distance a b =
    abs (a.X - b.X) + abs (a.Y - b.Y)

In [ ]:
let ruledOut row sensor =
    let sensorRange = distance sensor.Sensor sensor.Beacon
    let distanceToRow = abs (row - sensor.Sensor.Y)
    let reachIntoRow = sensorRange - distanceToRow
    if  reachIntoRow < 0 then
        Set.empty
    else
        [(sensor.Sensor.X - reachIntoRow) .. (sensor.Sensor.X + reachIntoRow)]
        //|> List.map (fun x -> { X = x; Y = row })
        |> Set.ofSeq

let combineRuledOut row sensors =
    let eliminated =
        sensors
        |> Seq.fold (
            fun set sensor ->        
                ruledOut row sensor
                |> Set.union set
            ) Set.empty

    let withoutBeacons =
        sensors
        |> Seq.fold (
            fun set s ->
                if s.Beacon.Y = row then
                    Set.remove s.Beacon.X set
                else set
            ) eliminated
    
    withoutBeacons

In [ ]:
#r "nuget: FsUnit"

open FsUnitTyped

let testInput =
    [|
        "Sensor at x=2, y=18: closest beacon is at x=-2, y=15"
        "Sensor at x=9, y=16: closest beacon is at x=10, y=16"
        "Sensor at x=13, y=2: closest beacon is at x=15, y=3"
        "Sensor at x=12, y=14: closest beacon is at x=10, y=16"
        "Sensor at x=10, y=20: closest beacon is at x=10, y=16"
        "Sensor at x=14, y=17: closest beacon is at x=10, y=16"
        "Sensor at x=8, y=7: closest beacon is at x=2, y=10"
        "Sensor at x=2, y=0: closest beacon is at x=2, y=10"
        "Sensor at x=0, y=11: closest beacon is at x=2, y=10"
        "Sensor at x=20, y=14: closest beacon is at x=25, y=17"
        "Sensor at x=17, y=20: closest beacon is at x=21, y=22"
        "Sensor at x=16, y=7: closest beacon is at x=15, y=3"
        "Sensor at x=14, y=3: closest beacon is at x=15, y=3"
        "Sensor at x=20, y=1: closest beacon is at x=15, y=3"
    |]

testInput
|> Array.map parseLine
|> combineRuledOut 10
|> Set.count
|> shouldEqual 26

In [ ]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_15.txt")
let sensors = 
    File.ReadAllLines(sourcePath)
    |> Array.map parseLine

let result = 
    sensors
    |> combineRuledOut 2000000
    |> Set.count

In [ ]:
printfn "Result: %d ruled out" result

## Part 2

In [ ]:
// Thanks to reddit users for pointing out that a unique solution must be just outside the range of the sensors
let limit = 4000000
let justOutside (s : Sensor) =
    let rad = 1 + distance s.Sensor s.Beacon
    [ 0 .. rad-1 ]
    |> List.collect (
        fun d -> 
            [
                { X = s.Sensor.X       + d; Y = s.Sensor.Y - rad + d }
                { X = s.Sensor.X + rad - d; Y = s.Sensor.Y       + d }
                { X = s.Sensor.X       - d; Y = s.Sensor.Y + rad - d }
                { X = s.Sensor.X - rad + d; Y = s.Sensor.Y       - d }
            ]
            |> List.filter (fun c -> c.X >= 0 && c.X <= limit && c.Y >= 0 && c.Y <= limit)
        )
    |> Array.ofList

let canBeSeenBy pos s =
    distance s.Sensor s.Beacon >= distance s.Sensor pos


In [ ]:
// It's also going to be on the border of more than one sensor
let testPixels =
    let pix = System.Collections.Concurrent.ConcurrentDictionary<Position, int>()
    for s in sensors do
        justOutside s
        |> Array.Parallel.iter (
            fun p ->
                pix.AddOrUpdate(p, (fun _ -> 1), (fun _ c -> c + 1))
                |> ignore
            )
    pix
    |> Seq.choose (fun kvp -> if kvp.Value > 1 then Some (kvp.Key, kvp.Value) else None)
    |> Array.ofSeq
    |> Array.sortByDescending snd


In [ ]:
let (pos,_) =
    testPixels 
    |> Array.find (
        fun (p,_) ->
            sensors |> Array.forall (fun s -> not (canBeSeenBy p s))
        )
printfn "Frequency:%d" (uint64 pos.X * uint64 limit + uint64 pos.Y)